# クラウドへ（そしてその先へ）
数値の問題を解決する方法については十分に検討しました。トレーニング部分をクラウドに移行します。（数値の問題ではこれ以上はローカルでの実行は不要ですが、他の問題については、ローカルでサブセットの問題をテストしてからクラウドに移動して全体を処理します）

いくつか設定しましょう。

最初にしなければならないことは、azureml.core パッケージがノートブック環境にインストールされているのを確認することです。Azure Notebooksを使用している場合は、簡単な2ステップのプロセスで確認できます。

## Azure Notebooks に依存関係を追加する
"Project Settings" をクリックします。

![Project Setings](https://raw.githubusercontent.com/sethjuarez/pytorchintro/master/images/project_settings.png)

次に、"Environment" タブでドロップダウンリストを左から順に `Requirements.txt` 、 `requirements.txt` 、 `Python 3.6` を選択します。

![Settings](https://raw.githubusercontent.com/sethjuarez/pytorchintro/master/images/settings.png)

これらのステップで、実行できるようになるはずです。

**注** もし上記の設定をしても問題が発生する場合は、Notebook でカーネルが Python 3.6 に設定されていることを確認してください。Python 3.6 になっていない場合は、次の操作で設定変更できます: ノートブックのメニューで [Kernel] -[Change Kernel] - [Python 3.6] を選択

In [2]:
import json, os, sys
import time
import azureml
from azureml.core.model import Model
from azureml.core import Workspace, Run, Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.dnn import PyTorch
#from azureml.widgets import RunDetails
#from torchvision import datasets, transforms

print("Azure ML SDK Version: ", azureml.core.VERSION)

ImportError: No module named 'azureml.core'

# Azure Machine Learning サービス（AzureML Service）のワークスペースを設定する

## AzureML Service ワークスペース を作成する
最初に必要な作業は、Azure Machine Learning サービス ワークスペースの作成です。
以下の Python コードから AzureML Service のワークスペースを作成します。

以下の Python スクリプトで
- `ワークスペース名`
- `リソースグループ名`
- `AzureサブスクリプションID` 
- `Azureのリージョン`

の4つの値を適宜設定して実行します。

ワークスペース名やリソースグループ名は、ご自身のAzure環境の中で区別できるものを任意で設定。
AzureサブスクリプションIDには、Azureポータルで左メニューから「サブスクリプション」を選び、一覧表示されるサブスクリプションの中でワークスペースを作成する先のものを選んでコピー＆ペーストします。
リージョンについては、AzureML Service が利用できるリージョンを設定します。

既に AzureML Service ワークスペース を作成済みでそれを利用したい場合は、そのワークスペースの情報を設定しても構いません。

スクリプトを実行すると `config.json` ファイルに設定が書き出されます。

In [3]:
config = {}
config["workspace_name"] = "decode2019_mlops"
config["resource_group"] = "decode2019"
config["subscription_id"] = "cd5e54ba-5b64-4acb-8ae5-72654d870add"
config["location"] = "southcentralus"

with open('config.json', 'w') as f:
    json.dump(config, f)

もし上記で設定した AzureML Service の ワークスペース に接続し、存在しない場合は新規に作成します。

In [4]:
from azureml.core.authentication import InteractiveLoginAuthentication

print("SDK Version:", azureml.core.VERSION)
#with open("aml_config/config.json") as f:
 #   config = json.load(f)

workspace_name = config["workspace_name"]
resource_group = config["resource_group"]
subscription_id = config["subscription_id"]
location = config["location"]

cli_auth = InteractiveLoginAuthentication()

try:
    ws = Workspace.get(
        name=workspace_name,
        subscription_id=subscription_id,
        resource_group=resource_group,
        auth=cli_auth
    )

except:
    # this call might take a minute or two.
    print("Creating new workspace...")
    ws = Workspace.create(
        name=workspace_name,
        subscription_id=subscription_id,
        resource_group=resource_group,
        # create_resource_group=True,
        location=location,
        auth=cli_auth
    )

# print Workspace details
print("\nWorkspace configuration succeeded. You are all set!")
print("Using workspace below;")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep="\n")

SDK Version: 1.0.33


get_workspace error using subscription_id=cd5e54ba-5b64-4acb-8ae5-72654d870add, resource_group_name=decode2019, workspace_name=decode2019_mlops


Creating new workspace...
Deploying KeyVault with name decode20keyvault16551d6c.
Deploying StorageAccount with name decode20storageb922e2386.
Deploying AppInsights with name decode20insights6ef874dc.
Deployed AppInsights with name decode20insights6ef874dc.
Deployed KeyVault with name decode20keyvault16551d6c.
Deployed StorageAccount with name decode20storageb922e2386.
Deploying Workspace with name decode2019_mlops.
Deployed Workspace with name decode2019_mlops.

Workspace configuration succeeded. You are all set!
Using workspace below;
decode2019_mlops
decode2019
southcentralus
cd5e54ba-5b64-4acb-8ae5-72654d870add


# クラウドコンピュート
次に、実験用のコンピュートターゲットを定義する必要があります。これは新規のワークスペースなので、クラスタの名前は自由に変更してください（私は 'racer' と呼んでいます）。以下のコードは自分のクラスタへの参照を取得しようとしますが、存在しない場合は作成します。クラスタを作成する場合、少し時間がかかります。また、予想外の課金をされないように、実験が完了したらクラスターをオフにしてください（実際には、min_node を 0 に設定して、長時間アイドル状態になるとクラスタが自動的にオフになる設定を検討してください）。 

**訳注** Azure の無償評価版などの GPU 最適化済みマシンを利用できない場合、またはコストを抑えたい場合は、vm_size を "STANDARD_D2_V2" にしてください。min_nodes を 1 以上にすると、訓練開始までの待ち時間を短縮できますが、コンピュートの削除し忘れなどで課金が継続されることがあるので注意してください。min_nodes を 0 にすると実行が終わると自動的にノードが削除されて課金されなくなります。

In [5]:
cluster = 'decode2019-MLOps'
try:
    compute = ComputeTarget(workspace=ws, name=cluster)
    print('Found existing compute target "{}"'.format(cluster))
except ComputeTargetException:
    print('Creating new compute target "{}"...'.format(cluster))
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', min_nodes=0, max_nodes=6)
    compute = ComputeTarget.create(ws, cluster, compute_config)
    compute.wait_for_completion(show_output=True)

Creating new compute target "decode2019-MLOps"...
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


# 実験の時間
コンピューティングターゲットが設定されたら、前回の小さなノートブックをリモートコンピューティング環境で実行できる単一のスクリプトにパッケージ化します。[あなたのために](train.py) train.py を作っておきました。実際、ファイルを見ると、前のノートブックから学んだものとまったく同じ概念がすべて表示されます（これはほとんどまったく同じですが、スクリプトへの受け渡しを容易にするために追加の事項を入れています）。

Azure ML サービスには実験という概念があります。実験ごとに複数回実行することができます。ここでは、実験の実行方法を定義する Estimator オブジェクトを使用しています。

### バックグラウンドで何をしてるか気にしないのであれば、ここは読む必要はありません
バックグラウンドでは、Estimator は基本的に実験を格納する docker イメージの定義です。このすべてについての最もよい部分は、あなたがあなたの実験に使うもの（TensorFlowのカスタムバージョンであっても他の何かであっても）に関係なく、それが必ず実行可能であるということです - 結局それはコンテナです。とても使いやすいです。

### 通常の手順に戻る
Estimator を Azure ML サービスで実行することを送信すると、現在のディレクトリの内容がコピーされ、新しいコンテナにまとめられます（それらは [.amlignore] ファイルに記述されたもの以外、全部アップロードされます）

また、'argparse' を使用しているので、推論器の定義の一部としてトレーニングスクリプトに外部パラメータを指定できます。

次の3行を実行して、何が起こるのか見てみましょう。

In [6]:
# 実験を作成
mnist = Experiment(ws, 'pytorchmnist')

# script parameters
script_params={
    '--epochs': 5,
    '--batch': 100,
    '--lr': .001,
    '--model': 'cnn'
}

# Estimator を作成
estimator = PyTorch(source_directory='.',
                       compute_target=compute, 
                       entry_script='train.py',
                       script_params=script_params,
                       use_gpu=True)

run = mnist.submit(estimator)

Submitting /home/nbuser/library/decode2019-Azure-MLOps directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


In [7]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
pytorchmnist,pytorchmnist_1557485074_a33bf082,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


Status が　`Queued`　になっていれば、キューにジョブが追加され、実行待ちの状態です。
以下の処理を起動すると、ジョブの現在の状態が表示されます。

表示は自動で更新されますので、ジョブの実行が開始され（Running）、完了（Completed）するまで数分ほど待ちましょう。

In [8]:
#RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

すべて完了すると、次のようになります:

![AzureML Run](https://raw.githubusercontent.com/sethjuarez/pytorchintro/master/images/run_widget.png)

実際に、損失関数は時間の経過とともに（平均して）減少し、モデルの精度が上がることに注意してください。learning_rate パラメータを変更して試してみてください。詳しくは、[Azure Machine Learning service でモデルのハイパーパラメーターを調整する](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters) を参照してください。

さて、どのようにしてこれらの素晴らしいチャートが表示できたのか疑問に思うかもしれません。これは Azure ML サービスが、実験結果に対して実用的な価値を付加してくれるところです。[いくつか](https://github.com/sethjuarez/pytorchintro/blob/master/train.py#L156-L166) の [戦略的](https://github.com/sethjuarez/pytorchintro/blob/master/train.py#L121-L122) に [配置](https://github.com/sethjuarez/pytorchintro/blob/master/train.py#L142-L143) されたログステートメントを使用して、Azure ML サービスはこの出力を作成しました。実際、値が複数回ログに記録されると、テーブル内の項目ではなくチャートが自動的に作成されます。

# モデル
トレーニングがすべて完了して出力が完了したら、実際に特定の実験のすべての実行の出力を確認し、それを「公式な」ワークスペースモデルに昇格させることができます。重要なファイル（つまり私たちをお金持ちにしてくれるかもしれないモデル）が通常 Jeff という名前のコンピュータ上に置かれるのは素晴らしい機能です。現在は、多くの人がモデルのバージョン管理さえしていませんが、以下のコードを実行してください。

In [9]:
run.get_file_names()

['azureml-logs/55_batchai_execution.txt',
 'azureml-logs/60_control_log.txt',
 'azureml-logs/80_driver_log.txt',
 'azureml-logs/azureml.log',
 'outputs/model.onnx',
 'outputs/model.pth']

In [10]:
model_file = 'outputs/model.pth'
run.download_file(name=model_file, output_file_path='model.pth')
model = Model.register(ws, model_name='PyTorchMNIST', model_path='model.pth', 
                       description='CNN PyTorch Model')

Registering model PyTorchMNIST


# イメージ
モデルが完成したので、それをプロダクションで使用する場合は、モデルの使用方法を定義する必要があります。これはスコアリングまたは推論とも呼ばれます。Azure ML サービスでは、基本的に2つのメソッドが必要です:
1. `init()`
2. `run(raw)` - JSON 文字列を取り込んで予測を返す

最初にスコアリングスクリプトが実行される環境を記述し、それを設定ファイルにまとめる必要があります。

In [11]:
myenv = CondaDependencies()
myenv.add_pip_package('numpy')
myenv.add_pip_package('torch')
with open('pytorchmnist.yml','w') as f:
    print('Writing out {}'.format('pytorchmnist.yml'))
    f.write(myenv.serialize_to_string())
    print('Done!')

Writing out pytorchmnist.yml
Done!


次に、Azure ML サービスにスコアリングスクリプトの場所を通知する必要があります。score.py を [あらかじめ作っておきました](score.py)。ファイルを見ると、init() メソッドと run(raw) メソッドの両方が簡単に見つかるはずです。ファイルをローカルで実行して、正しい動作をしていることを確認することもできます。

これですべてが完成したので、イメージを作成しましょう。

### バックグラウンドで何をしてるか気にしないのであれば、ここは読む必要はありません
基本的には、定義からdockerイメージを作成して、Workspace に表示される Azure Container Registry にプッシュします。

**注** しばらく時間がかかります

In [12]:
from azureml.core.image import ContainerImage, Image

# イメージの作成
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                runtime="python", 
                                conda_file="pytorchmnist.yml")

image = Image.create(ws, 'pytorchmnist', [model], image_config)
image.wait_for_creation(show_output=True)

Creating image
Running.................................................
SucceededImage creation operation finished for image pytorchmnist:1, operation "Succeeded"


# デプロイ
イメージ作成をせずに、残りの展開プロセスを Azure Pipelines のようなものに移動したいかもしれません。そうではなくて、このサービスを引き続きワークスペースにデプロイしたい場合は、以下を使用してください。

In [ ]:
from azureml.core.webservice import Webservice, AciWebservice

service_name = 'pytorchmnist-svc'

# check for existing service
svcs = [svc for svc in Webservice.list(ws) if svc.name==service_name]
if len(svcs) == 1:
    print('Deleting prior {} deployment'.format(service_name))
    svcs[0].delete()

# create service
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                            memory_gb=1, 
                                            description='simple MNIST digit detection')
service = Webservice.deploy_from_image(workspace=ws, 
                                    image=image, 
                                    name=service_name, 
                                    deployment_config=aciconfig)
service.wait_for_deployment(show_output=True)
print(service.scoring_uri)

イメージを ACI またはワークスペース Kubernetes クラスターにプッシュすることもできます。

時々うまくいかないことがあります・・・もし実行時にそうなったら、実際の [logs](deploy.log) を見てください。!

In [ ]:
with open('deploy.log','w') as f:
    f.write(service.get_logs())

# サービスの実行
以上でサービスは動作しています。適切に動作しているか見てみましょう。前から使っているテストデータをロードしてランダムな数字で試すことができます。

In [ ]:
#digits = datasets.MNIST('data', train=True, download=True,
#                        transform=transforms.Compose([
#                            transforms.ToTensor(),
#                            transforms.Lambda(lambda x: x.reshape(28*28))
#                        ]),
#                        target_transform=transforms.Compose([
#                            transforms.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, y, 1))
#                        ])
#                     )
#print(len(digits))

インデックスとして基本的に最大60,000まで任意の数を選ぶことができます。サービスがどのように動作しているかを見るために何回か試してみてください。

In [ ]:
#import torch
#from PIL import Image
#import matplotlib.pyplot as plt

#X, Y = digits[57435]
#X = X * 255
#plt.imshow(255 - X.reshape(28,28), cmap='gray')
#print(Y)

In [ ]:
# ポストしようとしているエンドポイントの場所
#image_str = ','.join(map(str, X.int().tolist()))
#print(image_str)

In [ ]:
#import json
#import requests
#service_url = service.scoring_uri
#print(service_url)
#r = requests.post(service_url, json={'image': image_str })
#r.json()

## 最後に
この小さな旅が参考になっていればうれしいです！ 私の目標は、機械学習の基本がそれほど悪いものではないと理解してもらうことです。コメント、提案、または分からないところは一言教えてください。